[View in Colaboratory](https://colab.research.google.com/github/NgoDinh/Data-science/blob/master/pu_learning_segmentation.ipynb)

In [0]:
import pandas as pd
import numpy as np
import psycopg2 as pg
from dateutil import parser
from dateutil.relativedelta import relativedelta
import time
import sys
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import datetime


start_time = time.time()

# function to query data from database
#=============================================================#
def query_data(target_date, date_end=False, positive = False):
      
    date_start = parser.parse(target_date) + relativedelta(days=-1)
    date_start = date_start.strftime('%Y-%m-%d')
    
    if date_end:
        date_end = date_end
    else:
        date_end = parser.parse(target_date) + relativedelta(days=1)
        date_end = date_end.strftime('%Y-%m-%d')
            
    extra_query = '''
    and so.partner_id in (select partner_id from sale_order where date_order between '%s' and '%s' 
                           UNION
                          select partner_id from pos_order where date_order between '%s' and '%s'
                            )
    '''%(date_start, date_end, date_start, date_end)
    
    if positive:
        extra = extra_query
        tmp = parser.parse(target_date) + relativedelta(days=-2)
        target_date = tmp.strftime('%Y-%m-%d')
    else:
        extra = ""
        #delete when launch
        tmp = parser.parse(target_date) + relativedelta(days=-2)
        target_date = tmp.strftime('%Y-%m-%d')
    
    get_total_customer='''
    Select
        tmp.id,
        tmp.customer_name,
        tmp.gioi_tinh_id,
        tmp.phone,
        tmp.time_life,
        tmp.check_info,
        date('%s')-date(max(tmp.create_date)) as last_time,
        sum(tmp.total_amount) as total_amount,
        sum(tmp.qty) as total_product,
        count(distinct tmp.name) as total_order,
        case 
        when 
            count(distinct tmp.name) = 1 then 9999
        else
            tmp.time_life / count(distinct tmp.name) 
        end as rebuy_time,
        sum(tmp.men) as total_men_product,
        sum(tmp.women) as total_women_product,
        sum(tmp.others) as total_y_product,
        count(distinct online) as online,
        count(distinct offline) as offline
    from
        (select 
            rp.id,
            rp.name customer_name,
            rp.gioi_tinh_id,
            rp.phone,
            date('%s') - date(rp.create_date) as time_life,
            so.create_date,
            Case
            When 
                (rp.birthdate_date is not null and rp.email !='' and (date('%s')-date(rp.birthdate_date)) < 36135) then 1
            else 
                0
            end as check_info,
            line.price_unit * line.product_uom_qty as total_amount,
            line.product_uom_qty as qty,
            so.name,pp.default_code,
            Case when upper(pp.default_code) like 'M%%' then line.product_uom_qty else 0 end men,
            Case when upper(pp.default_code) like 'W%%' then line.product_uom_qty else 0 end women,
            Case when (upper(pp.default_code) not like 'M%%' and upper(pp.default_code) not like 'W%%') then line.product_uom_qty else 0 end others,
            so.name online,
            null offline
        from
            sale_order as so
        inner join res_partner as rp on rp.id = so.partner_id
        inner join sale_order_line as line on line.order_id = so.id
        inner join product_product pp on pp.id = line.product_id
        INNER JOIN product_template as pt ON pt.id = pp.product_tmpl_id
        where
            rp.phone is not Null
            and (left(rp.phone,4) in ('0162','0163','0164','0165','0166','0167','0168','0169','0120','0121','0122','0126','0128','0123','0124','0125','0127','0129','0188','0186')
            or left(rp.phone,3) in ('086','096','097','098','090','093','091','094','092','088')
            or left(rp.phone,5) in ('08966'))
            and so.name not like '%%-%%'
            AND (so.section_id NOT IN (select id from crm_case_section where name like 'Dir%%') or so.section_id IS NULL)
            AND so.state != 'cancel'
            and pt.type NOT LIKE 'service%%'
            and (date('%s')-date(so.date_order))>0 %s
            
        UNION

        select 
            rp.id,
            rp.name customer_name,
            rp.gioi_tinh_id,
            rp.phone,
            date('%s') - date(rp.create_date) as time_life,
            so.create_date,
            Case
            When 
                (rp.birthdate_date is not null and rp.email !='' and (date('%s')-date(rp.birthdate_date)) < 36135) then 1
            else 
                0
            end as check_info,
            line.price_unit * line.qty as total_amount,
            line.qty,
            so.name,pp.default_code,
            Case when upper(pp.default_code) like 'M%%' then line.qty else 0 end men,
            Case when upper(pp.default_code) like 'W%%' then line.qty else 0 end women,
            Case when (upper(pp.default_code) not like 'M%%' and upper(pp.default_code) not like 'W%%') then line.qty else 0 end others,
            null online,
            so.name offline

        from 
            pos_order as so
        inner join res_partner as rp on rp.id = so.partner_id
        inner join pos_order_line as line on line.order_id = so.id
        inner join product_product pp on pp.id = line.product_id
        INNER JOIN product_template as pt ON pt.id = pp.product_tmpl_id
        where
            rp.phone is not Null
            and (left(rp.phone,4) in ('0162','0163','0164','0165','0166','0167','0168','0169','0120','0121','0122','0126','0128','0123','0124','0125','0127','0129','0188','0186')
            or left(rp.phone,3) in ('086','096','097','098','090','093','091','094','092','088')
            or left(rp.phone,5) in ('08966'))
            --and so.name not like '%%-%%'
            --AND (so.section_id NOT IN (select id from crm_case_section where name like 'Dir%%') or so.section_id IS NULL)
            AND so.state != 'cancel'
            and pt.type NOT LIKE 'service%%'
            and (date('%s')-date(so.date_order))>0 %s
            ) as tmp
    group by 1,2,3,4,5,6
    '''%(target_date,target_date,target_date,target_date,extra,target_date,target_date,target_date,extra)
    
    engine = pg.connect('postgresql://odoo:odoo@***/*')
    df = pd.read_sql(get_total_customer,engine)
    df['average_amount'] = df['total_amount']/df['total_order']
    df['average_product'] = df['total_product']/df['total_order']
    
    if positive:
        df['target'] = 1
    else:
        df['target'] = 0
    
    def check_gender(row):
        if np.isnan(row['gioi_tinh_id']) == False:
            val = row['gioi_tinh_id']
        else:
            if row['customer_name'].lower().startswith(('chú', 'chu', 'bac', 'bác', 'a')):
                val = 2
            elif row['customer_name'].lower().startswith('c'):
                val = 1
            else:
                val = 0
        return val

    df['gender'] = df.apply(check_gender, axis=1)
    df = df.drop(['customer_name', 'gioi_tinh_id'], axis=1)
    
    return df, get_total_customer
  
# get data by condition
#=============================================================#

 def get_data(target_date, date_end=False):
    
    ''' get data from database and then concat unlabled data to positive data'''
    
    start_time = time.time()
    data_u, query_u = query_data(target_date)
    target_date_revise = parser.parse(target_date) + relativedelta(years=-1)
    target_date_revise = target_date_revise.strftime('%Y-%m-%d')
    
    if date_end:
        date_end_revise = parser.parse(date_end) + relativedelta(years=-1)
        date_end_revise = date_end_revise.strftime('%Y-%m-%d')
    else:
        date_end_revise = date_end
        
    data_p, query_p = query_data(target_date_revise, date_end_revise, positive=True)
    
    def id_new(row):
        return str(row['id'])+'-'+str(row['target'])
    
    data_p['id_new'] = data_p.apply(id_new, axis=1)
    data_u['id_new'] = data_u.apply(id_new, axis=1)
    
    data_p = data_p.set_index('id_new')
    data_u = data_u.set_index('id_new')
    
    data_p = data_p.drop(['id'], axis=1)
    data_u = data_u.drop(['id'], axis=1)
#     data_low = data_u[data_u['id']<100000]
#     data_hight = data_u[data_u['id']>100000]
    
    data = pd.concat([data_u, data_p])
#     data_low = pd.concat([data_low, data_p])
#     data_hight = pd.concat([data_hight, data_p])
    
#     data_low = data_low.drop(['id'], axis=1)
#     data_hight = data_hight.drop(['id'], axis=1)
#     data_low = data_low.drop(['phone'], axis=1)
#     data_hight = data_hight.drop(['phone'], axis=1)
    
    phone_list = data['phone']
    data = data.drop(['phone'], axis=1)
    
    print(query_u)
    print('='*20)
    print(query_p)
    print(time.time() - start_time)

    return data, data_u, phone_list
  
# use pu_learning to train data
#=============================================================#
  
  def train_data(estimator, data, ratio=1):
    ''' If ratio is percentage of unlabed data have been used to train'''
    
    start_time = time.time()
    X = data.drop(['target'], axis=1)
    y = data['target']

    iP = y[y > 0].index
    iU = y[y <= 0].index
    if ratio > 1:
        n_estimators=ratio
    else:
        n_estimators = math.ceil(len(iU)/len(iP)*ratio)


    num_oob = pd.DataFrame(np.zeros(shape = y.shape), index = y.index)
    sum_oob = pd.DataFrame(np.zeros(shape = y.shape), index = y.index)

    for e in range(n_estimators):
        # Get a bootstrap sample of unlabeled points for this round
        ib = np.random.choice(iU, replace = True, size = len(iP))

        # Find the OOB data points for this round
        i_oob = list(set(iU) - set(ib))

        # Get the training data (ALL positives and the bootstrap
        # sample of unlabeled points) and build the tree
        Xb = X[y > 0].append(X.loc[ib])
        yb = y[y > 0].append(y.loc[ib])
        estimator.fit(Xb, yb)

        # Record the OOB scores from this round
        sum_oob.loc[i_oob, 0] += estimator.predict_proba(X.loc[i_oob])[:,1]
        num_oob.loc[i_oob, 0] += 1

        i = int((e+1)/n_estimators*100)
        sys.stdout.write('\r')
        # the exact output you're looking for:
        sys.stdout.write("[%-100s] %d%%" % ('='*(i),i))
        sys.stdout.flush()

    results = sum_oob / num_oob
    num_oob['score'] = sum_oob
    num_oob['result'] = results
    
    print("\n--- %s_%s seconds to Done with %s ---" %(str(estimator)[:1],time.time() - start_time, n_estimators))
    
    return num_oob 
  
# ensemble random forest and knn
#=============================================================#
  
  def ensemble_train_data(data, ratio, max_depth=3, n_neighbors=10, size=10000, knn_size=0.3):
    
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.ensemble import RandomForestClassifier
    
    knn = KNeighborsClassifier(n_neighbors=n_neighbors, n_jobs=-1)
    knn_result = train_data(knn, data, ratio = ratio)
    knn_result['tmp'] = knn_result.index
    knn_result['id'] = knn_result['tmp'].apply(lambda x: int(x[:-2]))
    knn_result = knn_result[(knn_result['id']<100000)&(knn_result['score']>0)]
    knn_result = knn_result.sort_values(['result'], ascending=False)
    knn_result = knn_result.head(round(size*knn_size))

    rc = RandomForestClassifier(max_depth=max_depth, n_jobs=-1, n_estimators=20)
    rc_result = train_data(rc, data, ratio = ratio)
    rc_result['tmp'] = rc_result.index
    rc_result['id'] = rc_result['tmp'].apply(lambda x: int(x[:-2]))
    rc_result = rc_result[(rc_result['id']>100000)&(rc_result['score']>0)]
    rc_result = rc_result.sort_values(['result'], ascending=False)
    rc_result = rc_result.head(round(size*(1-knn_size)))

    result = pd.concat([knn_result, rc_result])

#     writer = pd.ExcelWriter('E%s_%s_output_%s.xlsx'%(ratio, knn_size,datetime.datetime.now()))
#     result.to_excel(writer,'result')
#     knn_result.to_excel(writer,'knn_result')
#     rc_result.to_excel(writer,'rc_result')
#     writer.save()
    
    print("Done!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    
    return result, size

  
# function to test model
#=============================================================#
  
  def test_model(data_u, data_test, result, sample_size):
    
    start_time = time.time()
    data_test['id_tmp'] = data_test['id'].apply(lambda x: str(x) + "-" + str(0))
    model_score = 0
    for cus in result.index:
        if cus in data_test.id_tmp.values:
            model_score = model_score +1
            
    random_sample_score = []
    for _ in range(100):
        tmp = 0
        sample = data_u.sample(sample_size)
        for cus in sample.index:
            if cus in data_test.id_tmp.values:
                tmp = tmp + 1
        random_sample_score.append(tmp)

    random_sample_score = np.mean(random_sample_score)
#     random_sample_score = 24
    print(time.time() - start_time)
    
    return(model_score, random_sample_score)
  
    
# Run model
#=============================================================#

  data_test, query = query_data('2018-05-13', positive = True)
  data, data_u, phone_list = get_data('2018-05-13')
  result, size = ensemble_train_data(data, ratio=0.5, size=11000)
  print(test_model(data_u, data_test, result, size))